In [1]:
import os
import csv
from tqdm import tqdm
import pandas as pd
import networkx as nx
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv

# # 加载模型
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512").cuda()
model.eval()

# Get list of class names as column names
class_names = list(model.config.id2label.values())[1:]

def get_mask(image):
    inputs = feature_extractor(image, return_tensors="pt").to(model.device)
    # 对输入图片进行分割
    with torch.no_grad():    
        outputs = model(**inputs)
        logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
        predictions = torch.argmax(logits, dim=1).cpu().numpy().squeeze().astype('uint8')  # shape (batch_size, height/4, width/4)
       
    return predictions

def combine_graphs(all_predictions):
    G = nx.Graph()  # 创建空的大图
    all_nodes = []  # 创建空列表用于存储所有节点

    for predictions in all_predictions:
        mask = predictions
        labels = list(np.unique(predictions))

        # 创建空的小图
        current_graph = nx.Graph()

        # 添加节点到小图
        for label in labels:
            nodes = np.argwhere(mask == label)
            centroid = np.mean(nodes, axis=0)
            count = len(nodes)
            total_count = mask.size
            proportion = count / total_count * 100
            current_graph.add_node(
                label,
                centroid=centroid,
                label=class_names[label - 1],
                proportion=proportion,
            )

        all_nodes.extend(current_graph.nodes)  # 将节点添加到all_nodes列表中

        # 添加边到小图
        for i, (label_i, data_i) in enumerate(current_graph.nodes(data=True)):
            for j, (label_j, data_j) in enumerate(current_graph.nodes(data=True)):
                if i >= j:
                    continue
                dist = np.linalg.norm(data_i["centroid"] - data_j["centroid"])
                if dist < 42:
                    current_graph.add_edge(label_i, label_j)
                    
        # 将小图中的节点和边合并到大图中
        G.add_edges_from(current_graph.edges)

    G.add_nodes_from(all_nodes)  # 添加所有节点到大图

    return G

def get_embedding(G):
    # 获取节点数量
    num_nodes = G.number_of_nodes()

    # 定义嵌入层将特征从 1 维映射到 5 维
    in_channels = 5  # 输入特征维度
    hidden_channels = 16  # 隐层特征维度
    out_channels = 5  # 输出特征维度

    edge_index = from_networkx(G).edge_index

    data = Data(edge_index=edge_index, num_nodes=num_nodes)

    # 定义 GCN 模型
    class GCN(torch.nn.Module):
        def __init__(self, in_channels, hidden_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, hidden_channels)
            self.conv2 = GCNConv(hidden_channels, out_channels)

        def forward(self, edge_index):
            x = torch.randn(num_nodes, in_channels)  # 随机初始化节点特征
            x = F.relu(self.conv1(x, edge_index))
            x = self.conv2(x, edge_index)
            x = F.dropout(x, training=self.training)
            x = torch.mean(x, dim=0)  # 将所有节点的特征求平均得到图的嵌入向量
            x = torch.nn.Linear(x.shape[0], out_channels)(x)  # 将嵌入向量映射为输出特征维度
            return x

    # 初始化 GCN 模型并传入数据进行计算
    model = GCN(in_channels, hidden_channels, out_channels)
    embedding = model(data.edge_index)

    return embedding


data = pd.read_csv("E:/Dataset/GNN_Perception/wuhan_badu_SVI/name.csv")
df = pd.DataFrame(data)

folder_path = "E:/Dataset/GNN_Perception/wuhan_badu_SVI/baidu2023_pinjie"

for i in tqdm(range(3)):
    result = df[df['road_id'] == i]

    predict_set = []

    for idx in result.index:
        resultssss = result.at[idx, 'Name3']

        for filename in os.listdir(folder_path):
            file_id = filename.split('_')[2]
          
            if file_id in resultssss:
                image = Image.open(os.path.join(folder_path, filename))

                predict_set.append(get_mask(image))

    G = combine_graphs(predict_set)
    embedding = get_embedding(G)

    print(embedding)

d:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda3\envs\pytorch\lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
d:\anaconda3\envs\pytorch\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]

tensor([nan, nan, nan, nan, nan], grad_fn=<AddBackward0>)


 67%|██████▋   | 2/3 [00:04<00:02,  2.38s/it]

tensor([ 0.1760, -0.2687, -0.1686, -0.2684, -0.2596], grad_fn=<AddBackward0>)


100%|██████████| 3/3 [00:06<00:00,  2.09s/it]

tensor([-0.1501,  0.1295, -0.1963, -0.0760,  0.0125], grad_fn=<AddBackward0>)


### Data sample

![Alt text](image.png)![Alt text](image-1.png)